# WCS using Astrometry.net API
### Jazmin Garvey, 17/09/25

Below we have code to access the Astrometry.net website to submit large batches of files at a time. This means you can set it running and do other things! If you would like to alter this code to do something slightly different with the WCS solution, the documentation for using this API is here: https://astrometry.net/doc/net/api.html 

NOTE: Please do not submit more than ~50-100 images per day as the system gets overwhelmed and may block you. If this happens, please follow the instructions on URL.

In [3]:
## Imports:

import numpy as np
from astroquery.astrometry_net import AstrometryNet
from astropy.io import fits
import time
import os


Before you start, please go to https://nova.astrometry.net/signup and make a login (click login in the top corner). Then on the dashboard page click 'my profile'. Your api key should be on this page in green. Please don't share it with anyone else - its like a password! 

In the code below, words written in capital letters means you should put your own value there.

In [ ]:
## Setup:

ast = AstrometryNet()
ast.api_key = 'PASTE YOURS'

dir = 'HOME DIRECTORY FOR YOUR FILES'
solve_batch = []

for filename in os.listdir(dir):
    if filename.endswith('.fits'):   # you can add your own condition or just take your files from earlier in your notebook
        filepath = os.path.join(dir, filename)
        solve_batch.append(filename)


def solve_with_retry(filepath, max_retries=MAX RETRIES, base_timeout=TIMEOUT TIME):  # I use 120s and 3 tries

    for attempt in range(1, max_retries + 1):
        try:
            timeout = base_timeout * attempt  # increase timeout on retries to see if that was the issue
            print(f"Attempt {attempt}/{max_retries} for {filepath} with timeout {timeout}s")
            wcs_header = ast.solve_from_image(
                filepath,
                force_image_upload=True,
                solve_timeout=timeout
            )
            if wcs_header is not None:  # if we have a solution
                print(f"Sucessfully Solved {filepath}")
                return wcs_header
            else: # no solution
                print(f"{filepath} could not be solved.")
                return None
        except Exception as e: 
            print(f"'retries' is not dError on attempt {attempt} for {filepath}: {e}")
            if attempt < max_retries: # if we still have some tries left we will go again
                wait_time = 10 * attempt
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Solution failed after {max_retries} attempts for {filepath}") # if this fails, try increasing your timeout, or the image may just be unsolvable.
                return None


SyntaxError: invalid syntax. Perhaps you forgot a comma? (609612355.py, line 15)

Now we have set up our code to solve, I have written an example below on how to call the function and save the solution. The method I use (header.update and flush) saves all of the WCS information to the header of the file. This is like downloading the WCS header from the website and then saving that information into your header. 

Please note new keys are added to the header, and you can either print the header to see what these are or look it up online.

In [ ]:
## Running the code!

# B&C version:

results = {}
for file in solve_batch:
    wcs_header = solve_with_retry(file)
    results[f] = wcs_header
    
    # Save WCS back into the FITS header if solved
    if wcs_header is not None:
        with fits.open(file, mode='update') as hdulist:
            hdulist[0].header.update(wcs_header)
            hdulist.flush()  # write changes
        print(f"Solved file saved as {file}")
    else:
        print(f"No WCS for {file}")

# MOA version:

results = []
for file in solve_batch:
    wcs_header = solve_with_retry(file)
    results.append(wcs_header) # So you can have a list (or dictionary or whatever you like) with all of the wcs solutions in it. 
                               # You can extract the important values (CRPIX etc) from these headers instead, just like in the example.
                               # MOA headers are unable to be edited, so this is the next best thing - keep them seperate.